## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import optuna
import time
import lightgbm as lgb
import os

from catboost import CatBoostRegressor
from sklearn.metrics import root_mean_squared_error
from scipy.optimize import minimize as sp_minimize

d:\Program Files\conda_envs\nitor_kuas\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configs

In [2]:
train_path = '../data/train.csv'
test_path = '../data/test_for_participants.csv'
sample_path = '../data/sample_submission.csv'
VAL_START = '2025-06-01'
SEED = 42
N_TRIALS_LGB = 20
SAVED_LGB_PATH = '../models/lgb_final.txt'
ROUND_MULTIPLIER = 1.15

os.makedirs('../models', exist_ok=True)

## Data loading

In [3]:
train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)
sample_sub = pd.read_csv(sample_path)

In [4]:
for df in [train_raw, test_raw]:
    df['delivery_start'] = pd.to_datetime(df['delivery_start'])
    df['delivery_end'] = pd.to_datetime(df['delivery_end'])

train_raw['is_test'] = 0
test_raw['is_test'] = 1
test_raw['target'] = np.nan

df = pd.concat([train_raw, test_raw], ignore_index=True)
df = df.sort_values(['market', 'delivery_start']).reset_index(drop=True)
df

,id,target,market,global_horizontal_irradiance,diffuse_horizontal_irradiance,direct_normal_irradiance,cloud_cover_total,cloud_cover_low,cloud_cover_mid,cloud_cover_high,...,wind_speed_80m,wind_direction_80m,wind_gust_speed_10m,wind_speed_10m,solar_forecast,wind_forecast,load_forecast,delivery_start,delivery_end,is_test
0,0,-1.913,Market A,0.0,0.0,0.0,2.0,0.0,0.0,2.0,...,31.253719,245.501450,25.199999,15.077082,0.0,24050.1,38163.0100,2023-01-01 00:00:00,2023-01-01 01:00:00,0
1,5,-0.839,Market A,0.0,0.0,0.0,15.0,0.0,0.0,15.0,...,30.918108,242.241547,23.400000,14.186923,0.0,23886.3,37379.1898,2023-01-01 01:00:00,2023-01-01 02:00:00,0
2,10,-1.107,Market A,0.0,0.0,0.0,17.0,0.0,0.0,17.0,...,26.983196,224.999893,21.240000,12.413477,0.0,23366.5,36336.8303,2023-01-01 02:00:00,2023-01-01 03:00:00,0
3,15,0.035,Market A,0.0,0.0,0.0,16.0,0.0,0.0,16.0,...,22.218153,229.600174,16.199999,10.483357,0.0,22829.8,35337.7595,2023-01-01 03:00:00,2023-01-01 04:00:00,0
4,20,-0.829,Market A,0.0,0.0,0.0,10.0,0.0,0.0,10.0,...,27.210381,244.113022,18.359999,11.918120,0.0,22347.6,34474.3403,2023-01-01 04:00:00,2023-01-01 05:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145701,146752,NaN,Market F,0.0,0.0,0.0,100.0,100.0,100.0,100.0,...,24.863468,357.510498,27.359999,17.673029,0.0,9943.7,53190.5901,2025-11-30 18:00:00,2025-11-30 19:00:00,1
145702,146758,NaN,Market F,0.0,0.0,0.0,100.0,100.0,100.0,100.0,...,23.857979,354.805664,27.719999,16.700275,0.0,10235.4,54071.1413,2025-11-30 19:00:00,2025-11-30 20:00:00,1
145703,146764,NaN,Market F,0.0,0.0,0.0,100.0,100.0,100.0,71.0,...,21.485697,351.326904,25.559999,15.745627,0.0,10333.9,54517.9095,2025-11-30 20:00:00,2025-11-30 21:00:00,1
145704,146770,NaN,Market F,0.0,0.0,0.0,100.0,100.0,100.0,100.0,...,19.559633,353.659912,21.599998,14.471821,0.0,10214.1,54572.1696,2025-11-30 21:00:00,2025-11-30 22:00:00,1


## Feature engineering

In [5]:
ds = df["delivery_start"]
df["hour"]         = ds.dt.hour
df["day_of_week"]  = ds.dt.dayofweek
df["day_of_month"] = ds.dt.day
df["month"]        = ds.dt.month
df["quarter"]      = ds.dt.quarter
df["day_of_year"]  = ds.dt.dayofyear
df["year"]         = ds.dt.year
df["is_weekend"]   = (ds.dt.dayofweek >= 5).astype(np.int8)
df["week_of_year"] = ds.dt.isocalendar().week.astype(int)

In [6]:
market_map = {f"Market {c}": i for i, c in enumerate("ABCDEF")}
df["market_id"] = df["market"].map(market_map).astype(np.int8)


## Prepare X and y

In [7]:
observed_df = df[df['is_test'] == 0].copy()
test_df = df[df['is_test'] == 1].copy()

val_mask = observed_df['delivery_start'] >= VAL_START
train_df = observed_df[~val_mask]
val_df = observed_df[val_mask]

In [8]:
drop_cols = set(['id', 'target', 'market', 'delivery_start', 'delivery_end', 'is_test'])
feat_cols = sorted([c for c in df.columns if c not in drop_cols])
cat_idx = [feat_cols.index('market_id')]

X_train = train_df[feat_cols]
y_train_real = train_df['target'].values
y_train = np.arcsinh(train_df['target'].values)

X_val = val_df[feat_cols]
y_val_real = val_df['target'].values
y_val = np.arcsinh(val_df['target'].values)

X_all = observed_df[feat_cols]
y_all_real = observed_df['target'].values
y_all = np.arcsinh(observed_df['target'].values)

X_test = test_df[feat_cols]

## LightGbm baseline

In [9]:
print("Starting baseline evaluation...")
start_time = time.time()

# 1. The "Goldilocks" Hyperparameters
baseline_params = {
    "objective": "huber",     # Robust to the massive price spikes
    "alpha": 1.5,             # Moderate Huber threshold
    "metric": "rmse",
    "verbosity": -1,
    "seed": 42,               # STRICTLY FIXED for apples-to-apples comparison
    "n_jobs": -1,
    
    # Tree Structure (Moderate)
    "max_depth": 8,           # Deep enough for physics/weather interactions, shallow enough to stop overfitting
    "num_leaves": 127,        # 2^7 - 1 (Standard moderate size; your 980 leaves was massively overfitting)
    "min_child_samples": 50,  # Forces the model to generalize terminal nodes
    
    # Learning & Sampling
    "learning_rate": 0.05,    # Fast enough to train quickly, slow enough to learn smoothly
    "feature_fraction": 0.75, # Randomly hides 25% of features per tree so 'tightness_ratio' doesn't dominate every split
    "bagging_fraction": 0.8,  # Uses 80% of rows per tree to improve generalization
    "bagging_freq": 1,        # Perform bagging every iteration
    
    # Light Regularization
    "reg_alpha": 0.1,         # L1 regularization (Lasso)
    "reg_lambda": 0.1         # L2 regularization (Ridge)
}

# 2. Prepare Datasets
dataset_train = lgb.Dataset(X_train, y_train, categorical_feature=cat_idx, free_raw_data=False)
dataset_val = lgb.Dataset(X_val, y_val, reference=dataset_train, free_raw_data=False)

# 3. Train the Model (No Optuna)
model = lgb.train(
    baseline_params, 
    dataset_train, 
    num_boost_round=3000,     # High maximum rounds...
    valid_sets=[dataset_val],
    callbacks=[
        lgb.early_stopping(100, verbose=False), # ...but stops early if validation stops improving
        lgb.log_evaluation(200)                 # Prints progress every 200 rounds
    ],
)

# 4. Predict and Back-Transform
# (Predicting on X_val, then reversing the arcsinh transformation)
preds = model.predict(X_val)
real_preds = np.sinh(preds)

# 5. Evaluate True RMSE
final_rmse = root_mean_squared_error(y_val_real, real_preds)

print("\n" + "="*50)
print(f"✅ BASELINE RMSE: {final_rmse:.4f}")
print(f"   Iterations: {model.best_iteration}")
print(f"   Time taken: {time.time() - start_time:.0f}s")
print("="*50 + "\n")

# Optional: Print Top 10 features to verify your new feature is being used
imp = pd.Series(model.feature_importance("gain"), index=feat_cols)
print("Top 10 Features (Gain):")
print(imp.nlargest(10).round(0))

Starting baseline evaluation...
[200]	valid_0's rmse: 1.42092

✅ BASELINE RMSE: 40.7273
   Iterations: 215
   Time taken: 4s

Top 10 Features (Gain):
wind_speed_80m              297532.0
wind_forecast               294630.0
solar_forecast              132013.0
load_forecast               123376.0
market_id                   101502.0
wind_speed_10m               72655.0
day_of_year                  65639.0
surface_pressure             64691.0
year                         50501.0
direct_normal_irradiance     45082.0
dtype: float64


In [10]:
# Create a dataframe to analyze the validation results
val_results = val_df[['delivery_start', 'market_id']].copy()
val_results['actual'] = y_val_real
val_results['predicted'] = real_preds
val_results['month'] = val_results['delivery_start'].dt.month

print("\n" + "="*50)
print("📅 RMSE BY MONTH (VALIDATION SET)")
print("="*50)

# Calculate and print RMSE for each month in the validation set
for m in sorted(val_results['month'].unique()):
    month_data = val_results[val_results['month'] == m]
    month_rmse = root_mean_squared_error(month_data['actual'], month_data['predicted'])
    print(f"Month {m:02d} | Rows: {len(month_data):4d} | RMSE: {month_rmse:.4f}")

print("="*50 + "\n")


📅 RMSE BY MONTH (VALIDATION SET)
Month 06 | Rows: 4320 | RMSE: 28.6036
Month 07 | Rows: 4464 | RMSE: 44.3792
Month 08 | Rows: 4464 | RMSE: 46.4902



## LightGBM training

In [12]:
def obj_lgb(trial):
    params = {
        "objective": "huber",
        "alpha": trial.suggest_float("huber_alpha", 0.1, 3.0),
        "metric": "rmse",
        "verbosity": -1,
        "seed": SEED,
        "n_jobs": -1,
        "num_leaves": trial.suggest_int("num_leaves", 128, 1024),
        "learning_rate": trial.suggest_float("lr", 0.005, 0.05, log=True),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 200),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10, log=True),
        "max_depth": trial.suggest_int("max_depth", 6, 20),
    }

    dataset_train = lgb.Dataset(X_train, y_train, categorical_feature=cat_idx, free_raw_data=False)
    dataset_val = lgb.Dataset(X_val, y_val, reference=dataset_train, free_raw_data=False)
    
    model = lgb.train(
        params, dataset_train, num_boost_round=8000,
        valid_sets=[dataset_val],
        callbacks=[lgb.early_stopping(150), lgb.log_evaluation(0)],
    )
    
    preds = model.predict(X_val)
    real_preds = np.sinh(preds)

    rmse = root_mean_squared_error(y_val_real, real_preds)
    trial.set_user_attr("n_iter", model.best_iteration)
    
    return rmse

In [13]:
start_time = time.time()

study_lgb = optuna.create_study(
    direction="minimize", sampler=optuna.samplers.TPESampler(seed=SEED)
)
study_lgb.optimize(obj_lgb, n_trials=N_TRIALS_LGB)

lgb_rmse = study_lgb.best_value
lgb_iterations = study_lgb.best_trial.user_attrs["n_iter"]
lgb_best_params = study_lgb.best_params.copy()

if "lr" in lgb_best_params:
    lgb_best_params["learning_rate"] = lgb_best_params.pop("lr")

lgb_best_params.update({
    "objective": "huber",
    "metric": "rmse",
    "verbosity": -1,
    "seed": SEED,
    "n_jobs": -1})
print(f"-> Best val RMSE: {lgb_rmse:.4f} ({lgb_iterations} rounds, {time.time() - start_time:.0f}s)")

n_lgb = int(lgb_iterations * ROUND_MULTIPLIER)
dataset_all = lgb.Dataset(X_all, y_all, categorical_feature=cat_idx, free_raw_data=False)
lgb_final = lgb.train(lgb_best_params, dataset_all, num_boost_round=n_lgb)
lgb_test_preds = lgb_final.predict(X_test)

lgb_final.save_model(SAVED_LGB_PATH)
print(f"💾 LightGBM saved")


[I 2026-02-19 22:47:00,331] A new study created in memory with name: no-name-6632d188-7c3d-4214-a51a-09c65d5ad1d9


Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[937]	valid_0's rmse: 1.40202


[I 2026-02-19 22:47:20,552] Trial 0 finished with value: 40.76890389692428 and parameters: {'huber_alpha': 1.1861663446573514, 'num_leaves': 980, 'lr': 0.026975154833351143, 'min_child_samples': 124, 'feature_fraction': 0.5780093202212182, 'bagging_fraction': 0.5779972601681014, 'bagging_freq': 1, 'reg_alpha': 2.9154431891537547, 'reg_lambda': 0.2537815508265665, 'max_depth': 16}. Best is trial 0 with value: 40.76890389692428.


Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[7994]	valid_0's rmse: 1.42576


[I 2026-02-19 22:49:16,680] Trial 1 finished with value: 41.23938517891643 and parameters: {'huber_alpha': 0.1596950334578271, 'num_leaves': 998, 'lr': 0.033994812107955644, 'min_child_samples': 50, 'feature_fraction': 0.5909124836035503, 'bagging_fraction': 0.5917022549267169, 'bagging_freq': 3, 'reg_alpha': 0.12561043700013558, 'reg_lambda': 0.05342937261279776, 'max_depth': 10}. Best is trial 0 with value: 40.76890389692428.


...

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[911]	valid_0's rmse: 1.42034


[I 2026-02-19 23:00:59,777] Trial 18 finished with value: 40.63094190618325 and parameters: {'huber_alpha': 1.614367839064015, 'num_leaves': 559, 'lr': 0.011979275874070204, 'min_child_samples': 162, 'feature_fraction': 0.9292066062055611, 'bagging_fraction': 0.9945446963325211, 'bagging_freq': 2, 'reg_alpha': 0.0370048026225485, 'reg_lambda': 0.5093874271503771, 'max_depth': 16}. Best is trial 13 with value: 39.921128535324655.


Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[2435]	valid_0's rmse: 1.41863


[I 2026-02-19 23:02:36,414] Trial 19 finished with value: 40.453163837631365 and parameters: {'huber_alpha': 0.8200704166931959, 'num_leaves': 378, 'lr': 0.007018290879781808, 'min_child_samples': 15, 'feature_fraction': 0.99951815379155, 'bagging_fraction': 0.9346883490657583, 'bagging_freq': 5, 'reg_alpha': 8.418841767904226, 'reg_lambda': 0.06103726304159707, 'max_depth': 20}. Best is trial 13 with value: 39.921128535324655.


-> Best val RMSE: 39.9211 (1067 rounds, 936s)
💾 LightGBM saved


In [14]:
imp = pd.Series(lgb_final.feature_importance("gain"), index=feat_cols)
print("  Top-10 features:")
for f, v in imp.nlargest(10).items():
    print(f"    {f}: {v:.0f}")

  Top-10 features:
    wind_speed_80m: 1815100
    wind_forecast: 1692689
    solar_forecast: 977974
    load_forecast: 840568
    market_id: 622799
    day_of_year: 356796
    wind_speed_10m: 316585
    surface_pressure: 313870
    year: 241774
    direct_normal_irradiance: 231110
